# Data Extraction
Using the library pypdf extract the text from the exam that it is a pdf file

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader

Computing the expected total number of rows based in the number of questions and options

In [2]:
year = 2024
num_q = 210
num_o = 4
total_num_rows = num_q + (num_q*num_o)

## Extracting Questions from Pdf file

In [ ]:
path = f"data/Raw/Raw_Cuaderno_{year}_BIOLOGÍA_0_C.pdf"
reader = PdfReader(path)
full_text = []
for n in range(2, len(reader.pages)):
    page = reader.pages[n]
    text: str = page.extract_text()
    full_text += text.split("\n")

Creating a Data Frame of the text

In [4]:
exam_df = pd.DataFrame(full_text, columns=["text"])
exam_df.head()

,text
0,
1,Página: 1 de 19
2,1. La barrera hematoencefálica:
3,
4,1. Es permeable a todas las sustancias pre-


## Extracting Answers from .tsv file into a DataFrame

In [ ]:
answers_df = pd.read_table(f"data/Raw/Raw_Cuaderno_{year}_BIOLOGÍA_0_C_Respuestas.tsv")
answers_df.head()

,V0,RC,V0.1,RC.1,V0.2,RC.2,V0.3,RC.3,V0.4,RC.4
0,1,2,43,1,85,2,127,4,169,3.0
1,2,2,44,4,86,1,128,1,170,1.0
2,3,3,45,3,87,2,129,2,171,2.0
3,4,1,46,3,88,4,130,3,172,2.0
4,5,2,47,1,89,4,131,1,173,1.0


# Data Transformation

## Exam pdf

Removing rows containing "Página" and rows that have empty spaces

In [ ]:
exam_df = exam_df[~exam_df["text"].str.contains("Página")]
exam_df = exam_df["text"].str.strip()
exam_df = exam_df.replace("", np.nan)
exam_df = exam_df.dropna()
exam_df = exam_df.reset_index(drop=True)
exam_df.head()

1816    2. Produce patología mediante la invasión
1817                         del tejido entérico.
1818    3. Su mecanismo de transmisión es fecal -
1819                                        oral.
1820     4. Puede dar lugar a brotes y epidemias.
Name: text, dtype: object

Funcition to join the lines 
* Checks if a line ends with "-" meaning that the word is truncated, so it removes the last caracter, appends the line below and deletes the appended line 
* Checks if the following line the the first caracter can be converted into an integer, meaning that it is either a Question or an option, and proceds to append the line below and deletes it

In [ ]:
def process_multi_line_str(df):
    i = 0
    while i < len(df) - 1:
        line = df.iloc[i]
        if i < len(df) -1  and line.endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

In [8]:
exam_df_concat = process_multi_line_str(exam_df)
exam_df_concat.head()

0                      1. La barrera hematoencefálica:
1    1. Es permeable a todas las sustancias present...
2                      2. Es permeable al O2 y al CO2.
3                         3. Es impermeable al etanol.
4                           4. Es impermeable al agua.
Name: text, dtype: object

Print the rows that do not end in "." or ":" to fix them

In [42]:
num_rows_incorrect_expected = len(exam_df_concat) - total_num_rows
num_rows_incorrect = exam_df_concat[~exam_df_concat.str.endswith((".", ":"))].count()
id_rows_incorrect = exam_df_concat[~exam_df_concat.str.endswith((".", ":"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect}")
print(f"Number of rows not ending with '.' of ':' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 10
Number of rows not ending with '.' of ':' = 10
[55, 235, 516, 559, 612, 703, 789, 810, 838, 872]


In [47]:
def print_rows_incorrrect(wrong_id: list, df):
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

In [12]:
def fix_incorrect(num_row, df):
    num_row = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

In [ ]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)

rows_fix = [55, 559, 516, 612, 703, 810, 838, 840, ]

Id to fix 55:
53    3. Estimula la secreción de progesterona y est...
54    4. Comienza a ser secretada tras la involución...
55       12. En la fase de despolarización rápida (fase
56    0) del potencial de acción cardíaco, el princi...
57                                             1. Ca2+.
Name: text, dtype: object
Id to fix 235:
233                                              2. ATP.
234                                    3. ARN mensajero.
235                                               4. ADP
236    48. ¿Qué mecanismo actúa para transmitir la se...
237    1. Reciben la señal de un neurotransmisor, cam...
Name: text, dtype: object
Id to fix 516:
514                                       3. Transición.
515                                       4. Traducción.
516    104. ¿Qué término describe mejor las siguiente...
517                                      1, SINE y Alu?:
518                          1. Elementos transponibles.
Name: text, dtype: object
Id to fix 559:
557    112. S

The expeted number of rows for an exam with 210 questions and 4 options per question is 210+(210*4) = {total_num_rows}
Print true if the number of rows is the expected

In [ ]:
exam_df_fixed = fix_incorrect(rows_fix, exam_df_concat)
df_correct_rows = len(exam_df_fixed) == num_rows_incorrect_expected
print(df_correct_rows)
if not df_correct_rows:
    raise(Warning)

1050


In [15]:
exam_df_fixed = exam_df_fixed.to_frame()
groups = pd.Series((exam_df_fixed.index // 5) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1
exam_df_pivot = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()
exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns={
    1: "Question",
    2: "Option 1",
    3: "Option 2",
    4: "Option 3",
    5: "Option 4"  # Si tienes 4 opciones, ajusta según corresponda
})
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

exam_df_pivot.head()

,Question,Option 1,Option 2,Option 3,Option 4
0,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,2. Es permeable al O2 y al CO2.,3. Es impermeable al etanol.,4. Es impermeable al agua.
1,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,2. Entre la aracnoides y la piamadre.,3. Lleno de sangre.,4. En el sistema nervioso periférico.
2,3. La endolinfa:,1. Rellena el laberinto óseo del oído interno.,2. Contiene baja concentración de K+.,3. Rellena el laberinto membranoso del oído in...,4. Contiene alta concentración de Na+.
3,4. Las células olfatorias humanas:,1. Son células nerviosas bipolares.,2. Son células nerviosas multipolares.,3. Son células nerviosas unipolares.,4. Son células epiteliales.
4,5. La esclerosis múltiple es una patología que...,1. Hipermielinización de los axones con pérdid...,2. Desmielinización de los axones con pérdida ...,3. Incremento en la resistencia eléctrica axonal.,4. Reducción en la capacitancia axonal.


## Answers tsv

In [13]:
columnas_base = ['V0', 'RC']
columnas_apiladas = ['V0.1', 'B1', 'V0.2', 'B2', '']

original_col = ["V0", "RC"]
duplicate_cols = ["V0.1", "RC.1", "V0.2", "RC.2", "V0.3", "RC.3", "V0.4", "RC.4"]

answers_df_list = [answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols = duplicate_cols[i:i+2]
    df_pair_col = answers_df[pair_cols].rename(columns={pair_cols[0]: "V0", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

answers_df_clean = pd.concat(answers_df_list, ignore_index=True)

print(answers_df_clean)

      V0   RC
0      1  2.0
1      2  2.0
2      3  3.0
3      4  1.0
4      5  2.0
..   ...  ...
205  206  NaN
206  207  4.0
207  208  4.0
208  209  1.0
209  210  2.0

[210 rows x 2 columns]


## Joining the DataFrames

In [18]:
clean_df = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
clean_df = clean_df.drop(columns=["V0"])
clean_df.head()

,Question,Option 1,Option 2,Option 3,Option 4,RC
0,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,2. Es permeable al O2 y al CO2.,3. Es impermeable al etanol.,4. Es impermeable al agua.,2.0
1,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,2. Entre la aracnoides y la piamadre.,3. Lleno de sangre.,4. En el sistema nervioso periférico.,2.0
2,3. La endolinfa:,1. Rellena el laberinto óseo del oído interno.,2. Contiene baja concentración de K+.,3. Rellena el laberinto membranoso del oído in...,4. Contiene alta concentración de Na+.,3.0
3,4. Las células olfatorias humanas:,1. Son células nerviosas bipolares.,2. Son células nerviosas multipolares.,3. Son células nerviosas unipolares.,4. Son células epiteliales.,1.0
4,5. La esclerosis múltiple es una patología que...,1. Hipermielinización de los axones con pérdid...,2. Desmielinización de los axones con pérdida ...,3. Incremento en la resistencia eléctrica axonal.,4. Reducción en la capacitancia axonal.,2.0


In [22]:
print(clean_df[clean_df.isnull().any(axis=1)])

                                              Question  \
197  198. ¿Qué ameba es indistinguible morfológicam...   
199  200. La infección cerebral humana por Toxoplas...   
201  202. Los canales de Na+ pueden bloquearse sele...   
205             206. La ARN polimerasa III transcribe:   

                               Option 1                  Option 2  \
197           1. Entamoeba histolytica.   2. Entamoeba hartmanni.   
199  1. Coriorretinitis no pigmentaria.          2. Microcefalia.   
201                   1. Trietilamonio.       2. Benzodiazepinas.   
205                   1. ARN mensajero.  2. ARN de transferencia.   

                                              Option 3            Option 4  RC  
197                          3. Entamoeba moshkovskii.  4. Entamoeba coli. NaN  
199  3. Lesión ocupante de espacio, con anillo de c...    4. Hidrocefalia. NaN  
201                                     3. Verapamilo.       4. Lidocaína. NaN  
205                      3. Todos los A

In [23]:
clean_df_2 = clean_df.fillna(0)
clean_df_2.isnull().sum()

Question    0
Option 1    0
Option 2    0
Option 3    0
Option 4    0
RC          0
dtype: int64

In [26]:
clean_df_2["RC"] = clean_df_2["RC"].astype(int)
clean_df_2.dtypes

Question    object
Option 1    object
Option 2    object
Option 3    object
Option 4    object
RC           int32
dtype: object

# Data Loading

## Saving in csv file

In [30]:
clean_df_2["year"] = 2024
bir_2024 = clean_df_2

bir_2024.to_csv("data/clean_bir_2024.csv", index= False)

## Writing into SQL db